In [1]:
import pandas as pd 
import numpy as np 
import time
import re 

In [2]:
# Columns to read in
fields = ['Order Number', 'Product', 'Product Category','Client City','Sale Date Time','Discount Amount']
start = time.time()
data = pd.read_csv('data/original_data.csv',sep=';', usecols=fields) # NOTE: Remove nrows for final
print(f'Finished reading in dataset in {time.time()-start:.2f} seconds')

Finished reading in dataset in 30.90 seconds


In [3]:
# Rename columns
data.rename(columns=lambda x: x.replace(' ', '_').lower(), inplace=True)

In [4]:
# lambda to clear numeric values and remove whitespace
clean = lambda x: re.sub('[0-9]+', '', x).strip().lower()

# lowercase product and remove numbers
data['product'] = data['product'].apply(clean)
# lowecase category and remove numbers
data['product_category'] = data['product_category'].apply(clean)

# Clean up city name
data['client_city'] = data['client_city'].apply(lambda x: x.lower().strip().replace(' ','_'))

# Date formatter lambda
date_format = lambda x: x.split(' ')[0].replace('-','')
# Add unique basket ID column
data['basket_id'] =  data.order_number.apply(str) + data.sale_date_time.apply(date_format)

In [5]:
data.head()

,order_number,product,product_category,client_city,sale_date_time,discount_amount,basket_id
0,12,special gasoline,fuel,no_city,2017-03-31 04:10:00,0.0,1220170331
1,21,special diesel,fuel,no_city,2017-03-31 04:13:00,0.0,2120170331
2,38,special diesel,fuel,no_city,2017-03-31 04:25:00,0.0,3820170331
3,39,lubricant,lubricant,no_city,2017-03-31 04:26:00,0.0,3920170331
4,39,diesel auto clean,fuel,no_city,2017-03-31 04:26:00,0.0,3920170331


In [6]:
products = data.product_category.unique()
len(products)

39

In [7]:
import csv
with open('data/output/unique_products.csv', 'w+') as f:
    writer = csv.writer(f)
    writer.writerow(['PRODUCT']) # Adding header row since Rust ignores first row
    writer.writerows([[p] for p in products])

In [8]:
columns_to_export = ['product', 'product_category', 'client_city', 'discount_amount', 'basket_id']

In [9]:
data.to_csv('data/output/original_cleaned.csv', index=False, columns=columns_to_export)

### Export dataset without fuel

In [10]:
no_fuel = data[data.product_category != 'fuel']
no_fuel.to_csv('data/output/original_cleaned_no_fuel.csv', index=False, columns=columns_to_export)